In [4]:
pip install --upgrade MDAnalysis[analysis]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.6/46.6 kB 414.9 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 982.2 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 29.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 1.9 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 37.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.1/385.1 kB 9.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 873.3 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install --upgrade MDAnalysisTests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 MB 19.8 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 6.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 9.7 MB/s eta 0:00:00:00:01
  Created wheel for MDAnalysisTests: filename=MDAnalysisTests-2.7.0-py3-none-any.whl size=56061958 sha256=8db858fce38b0db0e57651e80824cbef7ae8588903bc340caacd7ba3400fac7e
  Stored in directory: /home/codespace/.cache/pip/wheels/32/66/0a/3d0bf05683e4b2e9a86f5a56445228d642f33d1e933c8ea397
Successfully built MDAnalysisTests
Note: you may need to restart the kernel to use updated packages.


In [6]:
import MDAnalysis as mda
from MDAnalysis import Universe
from MDAnalysis.tests.datafiles import PDB, GRO, XTC,TPR
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from numpy.linalg import norm
from numpy import array
import matplotlib.pyplot as plt
from MDAnalysis.analysis import distances
import MDAnalysis.lib.mdamath as math_help

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
#Calculating the number of crystalline molecules around a given molecule 
def calc_nc(all_grp,res_num,cutoff):
    """this function calculates the number of crystalline molecules,
        it requires the residues of all the molecules (all_grp),
        res_num is a specific residue number around which you wish to calculate crystalline molecules,
        cutoff specifies the radial distance within which number of molecules are estimated,
        nc gives the desired output"""
    g1=all_grp.select_atoms('resid {0}'.format(res_num))
    com_1=g1.center_of_mass(compound='residues')
    g2=all_grp.select_atoms('resid {0}-{1}'.format(1,res_num-1),'resid {0}-{1}'.format(res_num+1,all_grp.resnums[-1]))
    com_2=g2.center_of_mass(compound='residues')
    dist=(distances.distance_array(com_1,com_2,all_grp.dimensions))/10 # distances are converted in nm when divided by 10
    nc=0
    for i in dist[0]:
        if i<cutoff:
            nc+=1
    return(nc)            

In [ ]:
optimum_nc=calc_nc(all_grp,res_num,cutoff)

In [2]:
def local_density(res_num_i,sd):
    """ this function calculates local density contribution from a given molecule(res_num_i)
     sd stands for standard deviation associated with nc  """
    nc=calc_nc(all_grp,res_num_i,cutoff)
    if calc_nc(res_num_i)<=optimum_nc:
        local_density=math.exp(-(nc-optimum_nc)**2/(2*(sd**2)))
    else:
        local_density=1
    return(local_density)        

In [ ]:
def local_order(vector_1,vector_2,theta_0,theta_1,sd_0,sd_1):
    """this function calculates the local_order for a given residue
    theta_0 and theta_1 are the angles most commonly observed in the crystal lattice for a specific bond vector
    sd_0 and sd_1 are the correspomding standard deviations associated with the gaussian distributions of theta_0 and theta_1"""
    angle_radians=math_help.angle(vector_2[0],vector_1[0])
    local_order_contri=math.exp(-(angle_radians-theta_0)**2/(2*(sd_0**2))) + math.exp(-(angle_radians-theta_1)**2/(2*(sd_1**2)))
    return(local_order_contri)

---Main block starts here---

In [ ]:
#num_crystalline stores the total number of crystalline molecules for entire trajectory
num_crystalline=[]
for ts in all_grp.trajectory:
  contribution=0
  for i in f.residues:
    g1=all_grp.select_atoms('resid {0}'.format(i.resid))
    c1=g1.center_of_mass(compound='residues')
    g2=all_grp.select_atoms('resid {0}-{1}'.format(1,i.resid-1),'resid {0}-{1}'.format(i.resid+1,456))
    c2=g2.center_of_mass(compound='residues')
    dist_arr=distances.distance_array(c1,c2,all_grp.dimensions)/10 
    #defining the vector along carbonyl bond for the chosen residue
    v1_atom_1=all_grp.select_atoms('resid {0} and name C'.format(i.resid))
    v1_atom_2=all_grp.select_atoms('resid {0} and name O'.format(i.resid))
    v1=v1_atom_2.positions-v1_atom_1.positions
    nc=0
    counter=1
    local_order=0
    for j in dist_arr[0]:
      if j<cutoff:
        v2_atom_1=f.select_atoms('resid {0} and name C'.format(counter))
        v2_atom_2=f.select_atoms('resid {0} and name O'.format(counter))
        v2=v2_atom_2.positions-v2_atom_1.positions
        local_order_contri=local_order_contri(v1,v2,theta_0,theta_1,sd_angle_0,sd_angle_1)
        local_order=local_order+local_order_contri
      counter+=1
    if local_density(i,sd_num)!=0:
      local_order=local_order/local_density()
    local_density=local_density(i,sd_num)
    contribution=contribution+(local_density*local_order)
  num_crystalline.append(contribution)

In [1]:
all_grp=mda.Universe(f1,f2)
#f1 is a .tpr file(the topology file) 
#f2 is a .gro/.xtc file


NameError: name 'mda' is not defined

In [ ]:
# the time of the frames is stored in timestep
timestep=[]
for ts in all_grp.trajectory[1:400]:
  timestep.append(ts.time/1000)

In [ ]:
#plot of number of crystalline molecules as a function of time
plt.plot(timestep,num_crystalline)
plt.xlabel("time(ns)",fontsize=15,weight="bold")
plt.ylabel("Nc",fontsize=15,weight="bold")
plt.grid()
plt.show()